## PREPROCESAMIENTO DE DATOS

In [ ]:
import pandas as pd

train = pd.read_csv("C:\\Users\\USUARIO\\OneDrive - Fundación Universitaria San Pablo CEU\\Documentos\\IA\\competicion_ml_ia\\p3\\data\\public_train.csv")
train.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8310 entries, 0 to 8309
Columns: 1942 entries, ccs to fgp_620
dtypes: float64(1279), int64(662), object(1)
memory usage: 123.1+ MB
ccs        0
adduct     0
desc_1     0
desc_2     0
desc_3     0
          ..
fgp_616    0
fgp_617    0
fgp_618    0
fgp_619    0
fgp_620    0
Length: 1942, dtype: int64


In [27]:
test = pd.read_csv("C:\\Users\\USUARIO\\OneDrive - Fundación Universitaria San Pablo CEU\\Documentos\\IA\\competicion_ml_ia\\p3\\data\\public_test.csv")
test.head()

,adduct,desc_1,desc_2,desc_3,desc_4,desc_5,desc_6,desc_7,desc_8,desc_9,...,fgp_611,fgp_612,fgp_613,fgp_614,fgp_615,fgp_616,fgp_617,fgp_618,fgp_619,fgp_620
0,Monomer_[M+H],-2.52359,942.17682,1.29512,87.64198,0.16297,83.92964,84.34066,11.00000,4.80405,...,0,1,0,1,1,1,1,1,0,1
1,Monomer_[M-H],2.00538,12122.62988,1.80601,621.73560,0.07288,545.80798,554.61530,22.73333,5.64051,...,0,0,0,1,0,1,1,1,1,1
2,Monomer_[M-H],2.73984,6543.37451,1.52727,440.95099,0.08565,305.62857,313.87540,19.90000,5.31395,...,1,1,1,0,0,0,0,0,1,0
3,Monomer_[M+Na],-2.34683,695.94104,1.04344,74.62116,0.17652,65.28460,63.78675,10.81429,4.25346,...,1,0,0,1,1,0,0,0,1,1
4,Monomer_[M+H],0.00000,101.73023,0.03813,20.08101,0.35359,19.35335,20.59024,6.06667,2.62874,...,1,1,0,0,1,1,0,1,1,1


In [ ]:
from sklearn.preprocessing import OneHotEncoder
#Codificación de la columna adduct: Convierte la columna categórica adduct en variables numéricas usando técnicas como One-Hot Encoding o Label Encoding.
encoder = OneHotEncoder(sparse_output = False)
adduct_encoded = encoder.fit_transform(train[['adduct']])
train_encoded = pd.concat([train.drop(columns=['adduct']), pd.DataFrame(adduct_encoded)], axis=1)


In [33]:
from sklearn.preprocessing import StandardScaler
#Normalización/Estandarización: Normaliza las características numéricas (desc_x) para que tengan una escala uniforme.
scaler = StandardScaler()

# Asegúrate de que no se incluyan columnas no numéricas
X_train = scaler.fit_transform(train_encoded.drop(columns=['ccs']).values)



## ENTRENAMIENTO Y ESTIMACIÓN DE ERROR

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import median_absolute_error
from sklearn.feature_selection import SelectKBest, f_regression
import pandas as pd

# Asegúrate de que los nombres de las columnas sean cadenas
X = train_encoded.drop(columns=['ccs'])
y = train_encoded['ccs']

# Asegurar que X sea un DataFrame con columnas como cadenas
X.columns = X.columns.astype(str)

# División de los datos
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Rellenar valores nulos con la media
X_train = X_train.fillna(X_train.mean())
X_val = X_val.fillna(X_val.mean())

# Seleccionar las 100 mejores características
selector = SelectKBest(score_func=f_regression, k=100)
X_train = selector.fit_transform(X_train, y_train)
X_val = selector.transform(X_val)

# Entrenar el modelo
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Predicción y cálculo de MEDAE
y_pred = model.predict(X_val)
medae = median_absolute_error(y_val, y_pred)
print("MEDAE:", medae)



## GENERACIÓN DE PREDICCIONES

## CONCLUSIONES 